##### **Installing dependencies**

In [1]:
!pip install ipython-autotime gdown evaluate accelerate bitsandbytes peft loralib huggingface_hub transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 24.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


##### **Importing dependencies**

In [2]:
%load_ext autotime
import pandas as pd
import numpy as np
import nltk
import os
import zipfile
import tarfile
import re
import gdown
import gzip
import shutil
import wandb
import time
import torch
import psutil

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, precision_recall_fscore_support
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    RobertaTokenizerFast, 
    RobertaForSequenceClassification,
    GPT2TokenizerFast, 
    GPT2ForSequenceClassification,
    AlbertTokenizer, 
    AlbertForSequenceClassification,
    GenerationConfig,
    TrainingArguments,
    Trainer,
    pipeline,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    AdamW,
    get_scheduler
)
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
import evaluate
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    PeftConfig,
)
from huggingface_hub import login
import kagglehub

# from nltk.corpus import stopwords
# from nltk import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from google.colab import files
# from scipy.sparse import hstack
# from gensim.models import Word2Vec

import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message=".*clean_up_tokenization_spaces.*")
warnings.filterwarnings("ignore", message="Some weights of DistilBertForSequenceClassification were not initialized.*")
warnings.filterwarnings("ignore", message="Some weights of AlbertForSequenceClassification were not initialized.*")
warnings.filterwarnings("ignore", message=".*evaluation_strategy.*")
warnings.filterwarnings("ignore", message=".*gather along dimension 0.*")
warnings.filterwarnings("ignore", message=".*GradScaler.*")

time: 32.3 s (started: 2025-01-06 11:07:38 +00:00)


In [3]:
# Disable wandb Logging
os.environ["WANDB_MODE"] = "disabled"
wandb.init()

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda
time: 5.81 s (started: 2025-01-06 11:08:14 +00:00)


##### **Supporting functions**

In [4]:
def clean_review(review):
    review = re.sub(r'<.*?>', '', review)
    review = re.sub(r'http\S+|www\S+|https\S+', '', review, flags=re.MULTILINE)
    review = review.strip()
    return review

def preprocess_function(examples):
    inputs = tokenizer(examples["review"], truncation=True, padding=True, max_length=512)
    inputs["labels"] = [1 if label.lower() == "positive" else 0 for label in examples["sentiment"]]
    return inputs

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

time: 840 µs (started: 2025-01-06 11:08:21 +00:00)


##### **Loading data**

In [5]:
train_df_full = pd.read_csv("/kaggle/input/imdb-dataset-3/train.csv")
train_df = train_df_full.sample(n=3000, random_state=42)
train_df['review'] = train_df['review'].apply(clean_review)
train_df.reset_index(drop=True, inplace=True)

time: 1.03 s (started: 2025-01-06 11:08:28 +00:00)


In [6]:
test_df_full = pd.read_csv("/kaggle/input/imdb-dataset-3/test.csv")
test_df = test_df_full.sample(n=2000, random_state=42)
test_df['review'] = test_df['review'].apply(clean_review)
test_df.reset_index(drop=True, inplace=True)

time: 654 ms (started: 2025-01-06 11:08:29 +00:00)


In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

time: 83.4 ms (started: 2025-01-06 11:08:29 +00:00)


### **Experimentations for ALBERT - Phase 1:** keeping LoRA hyperparams fixed

In [8]:
model_checkpoint = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_checkpoint)
model = AlbertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Fixed LoRA parameters
rank = 8 
target_matrices = ["attention.query", "attention.key", "attention.value"]
lora_alpha = 16
lora_dropout = 0.1

# Changing hyperparams for batch size, epochs and learning rates
batch_sizes = [4, 8]
epochs_list = [3, 5]
learning_rates = [3e-5, 1e-4]

training_dropout = 0.1 # Fixed

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

time: 12.9 s (started: 2025-01-06 03:18:03 +00:00)


In [9]:
print(f"Model is running on device: {model.device}")

Model is running on device: cuda:0
time: 519 µs (started: 2025-01-06 03:18:23 +00:00)


In [10]:
results_phase_1 = []

for batch_size in batch_sizes:
    for epochs in epochs_list:
        for learning_rate in learning_rates:
            lora_config = LoraConfig(
                r=rank,
                lora_alpha=lora_alpha,
                target_modules=target_matrices,
                lora_dropout=lora_dropout,
                task_type="SEQ_CLS"
            )

            model_with_lora = get_peft_model(model, lora_config)
            
            start_time = time.time()
            print(f"\nRunning experiment with: Batch Size: {batch_size}, Epochs: {epochs}, Learning Rate: {learning_rate}")

            num_parameters = sum(p.numel() for p in model_with_lora.parameters())
            trainable_parameters = sum(p.numel() for p in model_with_lora.parameters() if p.requires_grad)
            trainable_percentage = (trainable_parameters / num_parameters) * 100
            
            print(f"Model has {num_parameters:,} total parameters")
            print(f"Model has {trainable_parameters:,} trainable parameters")
            print(f"{trainable_percentage:.2f}% of the parameters are trainable")

            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gpu_memory = torch.cuda.memory_allocated() / 1024**2  # in MB
                print(f"GPU memory allocated: {gpu_memory:.2f} MB")

            wandb.config.update({"model/num_parameters": model.num_parameters()}, allow_val_change=True)

            output_dir = f"./results_phase1_r{rank}_alpha{lora_alpha}_drop{lora_dropout}_targets{'_'.join(target_matrices)}_bs{batch_size}_epochs{epochs}_lr{learning_rate}"
            training_args = TrainingArguments(
                output_dir=output_dir,
                evaluation_strategy="epoch",
                learning_rate=learning_rate,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=epochs,
                weight_decay=0.01,
                save_total_limit=1,
                save_strategy="epoch",
                logging_dir="./logs",
                logging_steps=10,
                load_best_model_at_end=True,
            )

            trainer = Trainer(
                model=model_with_lora,
                args=training_args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_test,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
            )

            trainer.train()
            metrics = trainer.evaluate()

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Training time: {elapsed_time:.2f} seconds")

            results_phase_1.append({
                "Model": "ALBERT",
                "Batch Size": batch_size,
                "Epochs": epochs,
                "Learning Rate": learning_rate,
                "Rank": rank,
                "Alpha": lora_alpha,
                "LoRA Dropout": lora_dropout,
                "Target Matrices": target_matrices,
                "Accuracy": metrics["eval_accuracy"],
                "Precision": metrics["eval_precision"],
                "Recall": metrics["eval_recall"],
                "F1-Score": metrics["eval_f1"]                
            })


Running experiment with: Batch Size: 4, Epochs: 3, Learning Rate: 3e-05
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 44.73 MB


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.668600,0.670187,0.603500,0.624403,0.603500,0.577508
2,0.506800,0.531487,0.776000,0.783831,0.776000,0.775257
3,0.395000,0.440732,0.820500,0.821890,0.820500,0.820528


Training time: 646.55 seconds

Running experiment with: Batch Size: 4, Epochs: 3, Learning Rate: 0.0001
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 61.42 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.245200,0.305836,0.882500,0.883307,0.882500,0.882533
2,0.215700,0.293271,0.911000,0.911200,0.911000,0.910946
3,0.139600,0.291900,0.911000,0.910997,0.911000,0.910990


...

Training time: 635.88 seconds

Running experiment with: Batch Size: 8, Epochs: 5, Learning Rate: 3e-05
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 61.42 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.703000,0.686410,0.549000,0.548161,0.549000,0.539804
2,0.653600,0.645013,0.669500,0.672564,0.669500,0.669179
3,0.519100,0.527987,0.776000,0.779537,0.776000,0.775821
4,0.399800,0.420881,0.833000,0.833523,0.833000,0.833048
5,0.433100,0.393097,0.847500,0.847506,0.847500,0.847503


Training time: 1025.85 seconds

Running experiment with: Batch Size: 8, Epochs: 5, Learning Rate: 0.0001
Model has 11,723,524 total parameters
Model has 38,402 trainable parameters
0.33% of the parameters are trainable
GPU memory allocated: 61.42 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.516300,0.390825,0.840000,0.842739,0.840000,0.839954
2,0.321200,0.265708,0.904500,0.905465,0.904500,0.904525
3,0.211900,0.245570,0.911000,0.911200,0.911000,0.910946
4,0.150200,0.244345,0.914500,0.914637,0.914500,0.914517
5,0.278600,0.238338,0.919000,0.919143,0.919000,0.918960


Training time: 1026.58 seconds
time: 1h 52min 52s (started: 2025-01-06 03:18:28 +00:00)


In [11]:
# Testing evaluations saved
results_df_phase_1 = pd.DataFrame(results_phase_1)
results_df_phase_1.to_csv("7_FT_ALBERT_Experiments_FixedLoRA.csv", index=False)

time: 5.55 ms (started: 2025-01-06 05:12:32 +00:00)


### **Experimentations for ALBERT - Phase 2:** changing LoRA hyperparameters

In [8]:
model_checkpoint = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_checkpoint)
model = AlbertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Fixed parameters for batch size and epochs, etc
fixed_batch_size = 16
fixed_epochs = 5
fixed_learning_rate = 1e-4
training_dropout = 0.1

# LoRA parameter combinations
ranks = [8, 16]
target_matrices_list = [["attention.query"], ["attention.query", "attention.key"], ["attention.query", "attention.key", "attention.value"]]
lora_alpha = 16
lora_dropouts = [0.1, 0.2]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

time: 14.6 s (started: 2025-01-06 11:08:49 +00:00)


In [9]:
results_phase_2 = []

for rank in ranks:
    for target_matrices in target_matrices_list:
        for lora_dropout in lora_dropouts:
            lora_config = LoraConfig(
                r=rank,
                lora_alpha=lora_alpha,  # Fixed lora_alpha
                target_modules=target_matrices,
                lora_dropout=lora_dropout,
                task_type="SEQ_CLS"
            )

            model_with_lora = get_peft_model(model, lora_config)

            start_time = time.time()
            print(f"\nRunning experiment with: Rank: {rank}, Target Matrices: {target_matrices}, LoRA Dropout: {lora_dropout}")

            num_parameters = sum(p.numel() for p in model_with_lora.parameters())
            trainable_parameters = sum(p.numel() for p in model_with_lora.parameters() if p.requires_grad)
            trainable_percentage = (trainable_parameters / num_parameters) * 100
            
            print(f"Model has {num_parameters:,} total parameters")
            print(f"Model has {trainable_parameters:,} trainable parameters")
            print(f"{trainable_percentage:.2f}% of the parameters are trainable")

            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gpu_memory = torch.cuda.memory_allocated() / 1024**2  # in MB
                print(f"GPU memory allocated: {gpu_memory:.2f} MB")

            wandb.config.update({"model/num_parameters": model.num_parameters()}, allow_val_change=True)

            output_dir = f"./results_phase2_r{rank}_alpha{lora_alpha}_drop{lora_dropout}_targets{'_'.join(target_matrices)}_bs{fixed_batch_size}_epochs{fixed_epochs}_lr{fixed_learning_rate}"
            training_args = TrainingArguments(
                output_dir=output_dir,
                evaluation_strategy="epoch",
                learning_rate=fixed_learning_rate,
                per_device_train_batch_size=fixed_batch_size,
                per_device_eval_batch_size=fixed_batch_size,
                num_train_epochs=fixed_epochs,
                weight_decay=0.01,
                save_total_limit=1,
                save_strategy="epoch",
                logging_dir="./logs",
                logging_steps=10,
                fp16=True,
                load_best_model_at_end=True,
            )

            trainer = Trainer(
                model=model_with_lora,
                args=training_args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_test,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
            )

            trainer.train()
            metrics = trainer.evaluate()

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Training time: {elapsed_time:.2f} seconds")

            results_phase_2.append({
                "Model": "ALBERT",
                "Batch Size": fixed_batch_size,
                "Epochs": fixed_epochs,
                "Learning Rate": fixed_learning_rate,
                "Rank": rank,
                "Alpha": lora_alpha,  # Fixed alpha
                "LoRA Dropout": lora_dropout,
                "Target Matrices": target_matrices,
                "Accuracy": metrics["eval_accuracy"],
                "Precision": metrics["eval_precision"],
                "Recall": metrics["eval_recall"],
                "F1-Score": metrics["eval_f1"]
            })


Running experiment with: Rank: 8, Target Matrices: ['attention.query'], LoRA Dropout: 0.1
Model has 11,698,948 total parameters
Model has 13,826 trainable parameters
0.12% of the parameters are trainable
GPU memory allocated: 44.64 MB


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.692400,0.674462,0.583500,0.621132,0.583500,0.535353
2,0.620400,0.607277,0.687000,0.691465,0.687000,0.686398
3,0.496600,0.508597,0.760000,0.765200,0.760000,0.759561
4,0.484300,0.442865,0.807500,0.807750,0.807500,0.807284
5,0.415500,0.423513,0.815000,0.815000,0.815000,0.814918


Training time: 898.23 seconds

Running experiment with: Rank: 8, Target Matrices: ['attention.query'], LoRA Dropout: 0.2
Model has 11,698,948 total parameters
Model has 13,826 trainable parameters
0.12% of the parameters are trainable
GPU memory allocated: 61.67 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.693200,0.675770,0.580000,0.614674,0.580000,0.531958
2,0.635500,0.625485,0.672000,0.679291,0.672000,0.670483
3,0.517100,0.531594,0.744000,0.748592,0.744000,0.743600
4,0.498200,0.458264,0.798500,0.799679,0.798500,0.797957
5,0.419700,0.433823,0.817500,0.817781,0.817500,0.817295


...

Training time: 940.74 seconds

Running experiment with: Rank: 16, Target Matrices: ['attention.query', 'attention.key', 'attention.value'], LoRA Dropout: 0.1
Model has 11,760,388 total parameters
Model has 75,266 trainable parameters
0.64% of the parameters are trainable
GPU memory allocated: 62.47 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.583900,0.560658,0.729000,0.733980,0.729000,0.726332
2,0.301100,0.321680,0.867000,0.868315,0.867000,0.867025
3,0.244800,0.274331,0.894000,0.894043,0.894000,0.893964
4,0.274300,0.264743,0.895500,0.895521,0.895500,0.895472
5,0.254600,0.261431,0.896500,0.896521,0.896500,0.896473


Training time: 940.95 seconds

Running experiment with: Rank: 16, Target Matrices: ['attention.query', 'attention.key', 'attention.value'], LoRA Dropout: 0.2
Model has 11,760,388 total parameters
Model has 75,266 trainable parameters
0.64% of the parameters are trainable
GPU memory allocated: 62.61 MB


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.594100,0.573172,0.712000,0.717945,0.712000,0.708477
2,0.324400,0.329228,0.868000,0.868702,0.868000,0.868038
3,0.255500,0.280044,0.890000,0.890081,0.890000,0.889952
4,0.282300,0.269371,0.895500,0.895536,0.895500,0.895467
5,0.259500,0.265250,0.896500,0.896500,0.896500,0.896482


Training time: 940.79 seconds
time: 3h 5min 59s (started: 2025-01-06 11:09:23 +00:00)


In [10]:
# Testing evaluations saved
results_df_phase_2 = pd.DataFrame(results_phase_2)
results_df_phase_2.to_csv("7_FT_ALBERT_Experiments_FixedTrainingHyp.csv", index=False)

time: 12.3 ms (started: 2025-01-06 14:17:28 +00:00)
